In [ ]:
from openai import OpenAI
import pathlib, toml, os
import yaml, json
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
from jinja2 import Template

In [ ]:
config = toml.load("../../.streamlit/secrets.toml")
os.environ["OPENAI_API_KEY"] = config["openai"]["OPENAI_API_KEY"]

In [ ]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [ ]:
llm_client = OpenAI()

In [ ]:
model_name = 'all-mpnet-base-v2'

model = SentenceTransformer(
    model_name, 
    trust_remote_code=True,
    cache_folder="./models"   # explicitly setting cache location
)
emb_dimensions = model.get_sentence_embedding_dimension()

In [ ]:
history_storage = 'data/query_history.jsonl'
collection_name = "bfp-a3447q_v2"

In [ ]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=model.encode(query).tolist(),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
def search_with_history(query, limit=1, history_storage = history_storage):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=model.encode(query).tolist(),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )
    record = {}
    record['query']= query
    record['ground_truth_points']=[]
    record['limit']= limit
    record['result_points_scores']=[(point.id, point.score) for point in results.points]
    # Append without reading the whole file
    with open(history_storage, "a+") as f:
        f.write(json.dumps(record) + "\n")

    return results

In [ ]:
class PromptLoader:
    def __init__(self, path: str = "prompts.yaml"):
        with open(path, "r", encoding="utf-8") as f:
            self.prompts = yaml.safe_load(f)

    def render(self, name: str, **kwargs) -> str:
        """Render a named prompt with given variables."""
        template = Template(self.prompts[name])
        return template.render(**kwargs)

In [ ]:
def build_prompt(query, search_results):
    loader = PromptLoader("data/prompts.yaml")
    context = ""
    for index, payload in enumerate(search_results):
        context += f"{index}) Manual:\t{payload.payload['manual']},\nMain Chapter:\t{payload.payload['main_chapter']}\nChapter:\t{payload.payload['chapter']}\nContent: {payload.payload['content']}\n\n"
    return loader.render(
        "assistant_prompt",
        query=query,
        context=context
    )

In [ ]:
def refine_query(query):
    loader = PromptLoader("data/prompts.yaml")
    return loader.render(
        "refine_query",
        query=query
    )

In [ ]:
def llm(client, prompt, model='gpt-4.1-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt }]
    )
    return response.choices[0].message.content

In [ ]:
def rag(query, verbose_search= False, verbose_prompt = False):
    # llm_query = llm(llm_client,refine_query(query=query))
    llm_query = "What is the specific name of the connector used for connecting a power supply?"
    search_set = set()
    search_results = []
    queries = []
    queries.append(query)
    queries.append(llm_query)
    for q in queries:
        print("Query: ",q)
        results = search(q, 10)
        result_ids = set([p.id for p in results.points])
        unique_ids = result_ids - search_set
        for result in results.points:
            if result.id in unique_ids:
                search_results.append(result)
        search_set.update(result_ids)
    print(len(search_results), "results in total search\n")
        
    if verbose_search:
        print("Query search results:")
        print(*search_results, sep="\n\n")
    prompt = build_prompt(query,search_results)
    if verbose_prompt:
        print("Query prompt output:")
        print(prompt)
    message = llm(llm_client, prompt)
    return(message)

In [ ]:
query = "What is the name of connector, where we can plug in power supply?"

In [ ]:
llm_query = "What is the specific name of the connector used for connecting a power supply?"

In [ ]:
answer = rag(query, verbose_search = True)